# Evaluating the Causal Effect of Work Experience on Salary using text embeddings
## Overview

The use of text embeddings for Propensity Score Matching has been highlighted in several articles as a method to reduce confounding factors and measure causal effects.

In 2019, [Adapting text embeddings for causal inference](https://arxiv.org/abs/1905.12741) utilized text embeddings to compare article similarities, estimating the causal effect of gender on the popularity of articles.

In 2023, [Causal Matching with Text Embeddings: A Case Study in Estimating the Causal Effects of Peer Review Policies](https://aclanthology.org/2023.findings-acl.83/) applied a similar approach to estimate the impact of peer review policies on reviewer scores.

In reality, jobs with similar titles often vary significantly in actual duties, making it challenging to establish concrete salary standards, especially with the lack of transparency in salary data.

Accurate estimation of the influence of job roles on factors like years of experience, educational qualifications, and managerial requirements on actual salaries necessitates ensuring a high similarity in job content to avoid confounding factors.

This research focuses on the causal effect of whether having more than three years of work experience increases the likelihood of earning over 40K TWD per month.

The treatment is work experience > 1 years, and the outcome is a monthly income > 40K TWD. The most common PSM model used in this study involves converting job description text into embeddings and using Logistic Regression to estimate the treatment's propensity score.

After performing knn matching, the matching samples are used to estimate the treatment's causal effect using OLS.

In [89]:
import utils, json
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from psmpy import PsmPy
import statsmodels.api as sm 

In [3]:
bigquery_client = utils.get_bigquery_client("job")

In [28]:
query = """
with
  raw_data as 
    (
      select
        *,
        row_number() over(partition by id order by update_date desc) as order_id
      from ``{{project}}.{{dataset}}.{{datatable}}``
    ),
  latest as 
    (
      select
        * except(order_id)
      from raw_data
      where order_id = 1
    )
  select
    id,
    (work_exp not in ("不拘")) as gte_1_years,
    case
      when salary like "%待遇面議%" then true
      when salary like "%月薪%" then safe_cast(split(split(regexp_replace(salary, "月薪|年薪|,", ""), "~")[safe_offset(0)], "元")[safe_offset(0)] as int64) >= 40000
      when salary like "%年薪%" then safe_cast(split(split(regexp_replace(salary, "月薪|年薪|,", ""), "~")[safe_offset(0)], "元")[safe_offset(0)] as int64) >= 40000*12
      else null end as gte_40_k
  from latest
  where ( salary like any ("%面議%", "%月薪%", "%年薪%") ) 
    and ( salary not like any ("%部分%"))
    and lower(job_name) not like any ("%實習%", "%工讀%", "%打工%", "%接案%", "%parttime%", "%part time%", "%part-time%", "%intern%")
"""

job_detail = utils.get_bigquery_query(query, bigquery_client)

In [29]:
job = pd.read_csv("data/20231119_job_market_job.csv")
job_id = json.load(open("data/20231119_job_market_dsa_job_id.json", "r"))
job = job[job["id"].isin(job_id)]
job = pd.merge(job, job_detail, on = "id")

In [37]:
job_desc = job["job_desc"].tolist()

In [42]:
model_name = "all-MiniLM-L6-v2"

In [177]:
data_ols = job.filter(["gte_1_years", "id"]).copy()

In [182]:
pd.concat([data_ols.reset_index(drop=True), pd.DataFrame(embeddings).reset_index(drop=True)])

,gte_1_years,id,0,1,2,3,4,5,6,7,...,377,378,379,380,381,382,383,treatment,outcome,intercept
0,False,3gj1f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,NaN
1,False,6clt9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,NaN
2,True,6f1b4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,NaN
3,True,6vt4k,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,NaN
4,True,6vyyp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1940,<NA>,818n2,-0.098334,0.051626,0.056492,-0.011798,0.054378,0.074960,0.031375,-0.029579,...,0.005887,-0.054361,-0.003365,0.028539,0.056228,-0.003401,0.039778,True,False,1.0
1941,<NA>,81xt1,-0.086975,-0.004487,-0.049428,-0.041969,-0.036875,-0.078747,0.032857,0.077902,...,-0.013618,-0.013344,-0.010605,0.028825,0.083819,0.099260,0.048378,True,True,1.0
1942,<NA>,82f60,-0.035437,0.025614,-0.015352,0.002470,-0.042854,-0.054608,-0.020636,0.011532,...,-0.039057,0.011260,0.010996,0.024474,-0.090002,-0.025160,0.043698,True,True,1.0
1943,<NA>,83i87,-0.037037,-0.030697,-0.023439,-0.008578,-0.045396,-0.082691,-0.070045,0.034396,...,0.032513,-0.069372,0.051944,-0.003905,-0.003298,0.033563,0.024761,True,True,1.0


In [106]:
embeddings = pd.DataFrame(SentenceTransformer(model_name).encode(job_desc))
df = embeddings

psm = PsmPy(df, treatment = "treatment", indx = "id", exclude = [])
psm.logistic_ps(balance = True)
psm.knn_matched(matcher = "propensity_logit", replacement = False, caliper = None, drop_unmatched = True)
# psm.plot_match(Title = None, Ylabel = "Samples", Xlabel= "Propensity Score", names = ["Treatment: > 1 year", "Control: > 1 year"])
samples_ids = psm.matched_ids["id"].tolist() + psm.matched_ids["matched_ID"].tolist()

df["outcome"] = job["gte_40_k"]
df["intercept"] = 1
df = df[df["id"].isin(samples_ids)].filter(["outcome", "intercept", "treatment"]).reset_index(drop = True)

In [193]:
ols_model = sm.OLS(np.array(df.outcome, dtype=float), np.array(sm.add_constant(df.filter(["treatment"])), dtype=float)).fit()

summary = ols_model.summary2().tables[1]
summary = pd.concat([summary[summary.index == "const"].reset_index(), summary[summary.index == "x1"].reset_index()], axis=1)

summary["rsquared"] = ols_model.rsquared
summary["rsquared_adj"] = ols_model.rsquared_adj
summary["samples"]  = len(df.index)
summary["model_name"]  = model_name
summary

,index,Coef.,Std.Err.,t,P>|t|,[0.025,0.975],index,Coef.,Std.Err.,t,P>|t|,[0.025,0.975],rsquared,rsquared_adj,samples,model_name
0,const,0.770227,0.014787,52.088848,7.332213e-314,0.741217,0.799237,x1,0.126214,0.020912,6.035558,2.091937e-09,0.085187,0.16724,0.028674,0.027887,1236,all-MiniLM-L6-v2


# WIP